# Imports

In [1]:
import numpy as np
from dipy.io.gradients import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from bids.layout import BIDSLayout
import dipy.reconst.dti as dti
from dipy.io.image import load_nifti_data, load_nifti, save_nifti
from dipy.data import get_fnames, get_sphere
from dipy.core.sphere import HemiSphere
from dipy.segment.mask import median_otsu
import dipy.reconst.dti as dti
import nibabel as nib
from dipy.sims.voxel import single_tensor_odf
import numpy as np
import gudhi as gd
from dipy.reconst.rumba import RumbaSDModel
from dipy.reconst.csdeconv import auto_response_ssst
from fury import window, actor

# Init

In [2]:
gradient_layout = BIDSLayout("./openneuro/ds001907/sub-RC4201/ses-1/", validate=False)

subj = 'RC4201'

dwi_fname = gradient_layout.get(subject=subj, suffix='dwi', extension='.nii.gz', return_type='file')[0]
bvec_fname = gradient_layout.get( extension='.bvec', return_type='file')[0]
bval_fname = gradient_layout.get( extension='.bval', return_type='file')[0]

dwi_img = nib.load(dwi_fname)
affine = dwi_img.affine

bvals, bvecs = read_bvals_bvecs(bval_fname, bvec_fname)
gtab = gradient_table(bvals, bvecs)


#sphere = HemiSphere.from_sphere(get_sphere('repulsion724')) they recommend to use hemisphere for rumba to optimize time
sphere = get_sphere('repulsion724')

In [3]:
dwi_data = dwi_img.get_fdata()
maskdata, mask = median_otsu(dwi_data, vol_idx=range(10, 50), median_radius=3,
                             numpass=1, autocrop=True, dilate=2)

In [4]:
rumba = RumbaSDModel(gtab)
response, _ = auto_response_ssst(gtab, dwi_data, roi_radii=10, fa_thr=0.7)

(array([0.00165202, 0.00039875, 0.00039875]), 69502.95412529839)


In [ ]:
def dataToOdf(data):
    rumba_fit = rumba.fit(data)

    odf = rumba_fit.odf()
    combined = rumba_fit.combined_odf_iso

    fodf_spheres1 = actor.odf_slicer(combined, sphere=sphere, norm=True,
                                    scale=0.5, colormap=None)
    return fodf_spheres1,odf,combined

In [21]:
#global fit
rumba = RumbaSDModel(gtab, wm_response=response[0], gm_response=None,
                     voxelwise=False, use_tv=False, sphere=sphere)
data_tv = dwi_data[20:24, 42:46, 21:22]

vox = dwi_data[29:30,59:60,38:39]

# For now let's solve for voxel space
data1

In [22]:
rumba_fit = rumba.fit(data_tv)

odf = rumba_fit.odf()
combined = rumba_fit.combined_odf_iso

fodf_spheres1 = actor.odf_slicer(combined, sphere=sphere, norm=True,
                                scale=0.5, colormap=None)
scene = window.Scene()
scene.add(fodf_spheres1)
window.show(scene)

In [7]:
fodf_spheres = actor.odf_slicer(odf, sphere=sphere, norm=True,
                                scale=0.5, colormap=None)

scene = window.Scene()
scene.add(fodf_spheres)
window.show(scene)

In [8]:
fodf_spheres.faces.shape

# Slice All Odfs into singular Objects First?
# --- For now use the one sperated above

# Create Tetrahedra from Triangle Mesh
# This is only for one Singular Object.
faces = fodf_spheres.faces
print(faces.shape)
vertices = fodf_spheres.vertices 
x1, x2, y1, y2, z1, z2 = fodf_spheres.GetBounds()
center = fodf_spheres.GetPosition()
tetrahedra_list = []
for face in faces:
    # Get the three vertices that make up the face
    v1, v2, v3 = vertices[face[0]], vertices[face[1]], vertices[face[2]]
    # Create a list of all four vertices with the center of the ODF
    center = np.array(center)
    tetrahedra = [v1, v2, v3, center]
    # newFaces = Vertex index -> create 3 new faces 
    # Add the tetrahedron to the list
    tetrahedra_list.append(tetrahedra)

(1444, 3)
